In [4]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy import units as u

from astropy.io import fits

from astropy.nddata import Cutout2D
from astropy.wcs import WCS

from tqdm import tqdm

In [2]:
import mastcasjobs




def PS1cat(wcs):
    jobs = mastcasjobs.MastCasJobs(context="PanSTARRS_DR2")
    s = wcs.array_shape
    ra, dec = wcs.all_pix2world(s[0]/2,s[1]/2,0)
    r = ((1/2*np.sqrt(s[0]**2+s[1]**2) + 5) * 24)/60
    
    f = wcs.calc_footprint()
    
    ramin = min(f[:,0])
    ramax = max(f[:,0])
    decmin = min(f[:,1])
    decmax = max(f[:,1])

    queryMean = """select 
    o.raMean, o.decMean,o.iMeanPSFMag,o.zMeanPSFMag,o.iMeanKronMag,soa.iKronRad
    from fGetNearbyObjEq({ra},{dec},{rad}) x
    JOIN MeanObjectView o on o.ObjID=x.ObjId
    LEFT JOIN StackObjectAttributes AS soa ON soa.objID = x.objID
    WHERE o.nDetections>5
    AND soa.primaryDetection>0
    AND o.raMean > {ramin} and o.raMean < {ramax} and o.decMean > {decmin} and o.decMean < {decmax}
    AND o.iMeanPSFMag <19
    """.format(ra=ra,dec=dec,rad=r,ramin=ramin,ramax=ramax,decmin=decmin,decmax=decmax)

    meanResults = jobs.quick(queryMean, task_name="python mean table search").to_pandas()

    #data = [list(i) for i in meanResults]
    #columns = list(meanResults.columns)

    df = meanResults#pd.DataFrame(data=data,columns=columns)
    df = df.replace(-999,np.nan)
    return df



#select r.*
#from refcat2 as r
#inner join fGetObjFromRect(0,0.2,0,0.2) 

#from fGetNearbyObjEq(0,0,5.0/60.0) as n
#inner join refcat2 as r on n.objid=r.objid

#select ndistance as dstDegrees, r.*
#from fGetNearbyObjEq(0,0,5.0/60.0) as n
#inner join refcat2 as r on n.objid=r.objid
#order by n.distance
def Refcat(wcs):
    jobs = mastcasjobs.MastCasJobs(context="HLSP_ATLAS_REFCAT2")
    s = wcs.array_shape
    ra, dec = wcs.all_pix2world(s[0]/2,s[1]/2,0)
    r = ((1/2*np.sqrt(s[0]**2+s[1]**2) + 5) * 24)/60
    
    f = wcs.calc_footprint()
    
    ramin = min(f[:,0])
    ramax = max(f[:,0])
    decmin = min(f[:,1])
    decmax = max(f[:,1])

    queryMean = """select r.RA, r.Dec, r.Gaia, r.i, r.z
    from fGetNearbyObjEq({ra},{dec},{rad}) as n
    INNER JOIN refcat2 as r on n.objid=r.objid
    WHERE r.i < 13
    """.format(ra=ra,dec=dec,rad=r,ramin=ramin,ramax=ramax,decmin=decmin,decmax=decmax)

    meanResults = jobs.quick(queryMean, task_name="python mean table search").to_pandas()

    #data = [list(i) for i in meanResults]
    #columns = list(meanResults.columns)

    df = meanResults#pd.DataFrame(data=data,columns=columns)
    df = df.replace(-999,np.nan)
    return df


def Gaiacat(wcs):
    jobs = mastcasjobs.MastCasJobs(context="GAIA_EDR3")
    s = wcs.array_shape
    ra, dec = wcs.all_pix2world(s[0]/2,s[1]/2,0)
    r = ((1/2*np.sqrt(s[0]**2+s[1]**2) + 5) * 24)/60
    
    f = wcs.calc_footprint()
    
    ramin = min(f[:,0])
    ramax = max(f[:,0])
    decmin = min(f[:,1])
    decmax = max(f[:,1])

    queryMean = """select T.ra,T.dec,T.phot_g_mean_mag as gaia
    from fGetNearbyObjEq({ra},{dec},{rad}) as n
    join GAIApublicVOview T on T.source_id = n.objID
    WHERE T.phot_g_mean_mag < 14
    """.format(ra=ra,dec=dec,rad=r,ramin=ramin,ramax=ramax,decmin=decmin,decmax=decmax)

    meanResults = jobs.quick(queryMean, task_name="python mean table search").to_pandas()

    #data = [list(i) for i in meanResults]
    #columns = list(meanResults.columns)

    df = meanResults#pd.DataFrame(data=data,columns=columns)
    df = df.replace(-999,np.nan)
    return df

#    INNER JOIN refcat2 as r on n.objid=r.objid
#    WHERE r.i < 13
#AND o.raMean > {ramin} and o.raMean < {ramax} and o.decMean > {decmin} and o.decMean < {decmax}
#    AND o.iMeanPSFMag <14

In [12]:
file = 's0023_58939.901_0591_3.sw.fits'
image = fits.open(file)[0].data
hdu = fits.open(file)[0]
wcs = WCS(hdu)

In [9]:
image = fits.open(file)[0].data
hdu = fits.open(file)[0]


start = hdu.header['SCCSA']-1
end = hdu.header['SCCED']

number = 16
width = 2048/number
ps1 = None
gaia = None
for i in tqdm(range(number)):
    sr = i*width
    se = (i+1)* width
    for j in range(number):
        sc = start+j*width
        ec = start+(j+1)*width
        FFI_wcs = WCS(hdu.header)
        #Main_cutout = Cutout2D(image,(1024+start,1024),2024,wcs=FFI_wcs)
        rowcent = sr + (width/2 - 0.5)
        colcent = sc + (width/2 - 0.5)
        print(colcent,rowcent)
        temp = Cutout2D(image,(colcent,rowcent),width,wcs=FFI_wcs)
        wcs = temp.wcs
        p = PS1cat(wcs)
        g = Gaiacat(wcs)
        if ps1 is None:
            ps1 = p
            
            #print('added first region')
        else:
            ps1 = ps1.append(p)
            ps1 = ps1.drop_duplicates()
        
        if gaia is None:
            gaia = g
        else:
            gaia = gaia.append(g)
            gaia = gaia.drop_duplicates()
            #print('added next region')
            
ps1.to_csv('ps1_test_cat.csv',index=False)

  0%|          | 0/16 [00:00<?, ?it/s]

107.5 63.5
235.5 63.5


  0%|          | 0/16 [00:20<?, ?it/s]


KeyboardInterrupt: 

In [13]:
jobs = mastcasjobs.MastCasJobs(context="PanSTARRS_DR2")
s = wcs.array_shape
ra, dec = wcs.all_pix2world(s[0]/2,s[1]/2,0)
r = ((1/2*np.sqrt(s[0]**2+s[1]**2) + 5) * 24)/60

f = wcs.calc_footprint()

ramin = min(f[:,0])
ramax = max(f[:,0])
decmin = min(f[:,1])
decmax = max(f[:,1])

queryMean = """select 
o.raMean, o.decMean,o.iMeanPSFMag,o.zMeanPSFMag,o.iMeanKronMag,soa.iKronRad
from fGetNearbyObjEq({ra},{dec},{rad}) x
JOIN MeanObjectView o on o.ObjID=x.ObjId
LEFT JOIN StackObjectAttributes AS soa ON soa.objID = x.objID
WHERE o.nDetections>5
AND soa.primaryDetection>0
AND o.raMean > {ramin} and o.raMean < {ramax} and o.decMean > {decmin} and o.decMean < {decmax}
AND o.iMeanPSFMag <19
""".format(ra=ra,dec=dec,rad=r,ramin=ramin,ramax=ramax,decmin=decmin,decmax=decmax)


In [14]:
queryMean

'select \no.raMean, o.decMean,o.iMeanPSFMag,o.zMeanPSFMag,o.iMeanKronMag,soa.iKronRad\nfrom fGetNearbyObjEq(190.08143366297801,8.604570917416329,598.0060402378487) x\nJOIN MeanObjectView o on o.ObjID=x.ObjId\nLEFT JOIN StackObjectAttributes AS soa ON soa.objID = x.objID\nWHERE o.nDetections>5\nAND soa.primaryDetection>0\nAND o.raMean > 182.79923815699243 and o.raMean < 198.3243391375747 and o.decMean > 0.6145507340964471 and o.decMean < 16.14931915066824\nAND o.iMeanPSFMag <19\n'

In [15]:
queryMean = """select T.ra,T.dec,T.phot_g_mean_mag as gaia
    from fGetNearbyObjEq({ra},{dec},{rad}) as n
    join GAIApublicVOview T on T.source_id = n.objID
    WHERE T.phot_g_mean_mag < 14
    """.format(ra=ra,dec=dec,rad=r,ramin=ramin,ramax=ramax,decmin=decmin,decmax=decmax)

In [16]:
queryMean

'select T.ra,T.dec,T.phot_g_mean_mag as gaia\n    from fGetNearbyObjEq(190.08143366297801,8.604570917416329,598.0060402378487) as n\n    join GAIApublicVOview T on T.source_id = n.objID\n    WHERE T.phot_g_mean_mag < 14\n    '